In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

def nn2na(NN):
    # Get every location where exist an arc:
    idxs = np.argwhere(NN)
    # Preallocate NA matrix, dimension is (nodes, arcs)
    NA = np.zeros([NN.shape[0], idxs.shape[0]]).astype(int)
    C = np.zeros(NA.shape[1])
    # Loop in every arc, complete from (1) to (-1)
    for i, arc in enumerate(idxs):
        # Node arc:
        NA[arc[0], i] = 1 # From
        NA[arc[1], i] = -1 # To

    arc_idxs = [(arc[0], arc[1]) for arc in idxs]

    return NA, arc_idxs

def get_usage_string(arc_idxs, res_flow, capacity):
    return {arc: '%s/%s' % (flow, cap) for arc, flow, cap in zip(arc_idxs, res_flow, capacity)}

def get_min_cut(arc_idxs, np_res_flow, np_capacity):
    np_capacity = np.where(np_capacity == None, 999, np_capacity)

    idxs = np.argwhere((np_res_flow - np_capacity) == 0)
    return [arc_idxs[i[0]] for i in idxs]

#import the data Node Node Matrix
NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [1, 0, 0, 0, 0, 0]])

#formulation of LP:
Aeq, arc_idxs = nn2na(NN)
C = np.array([0, 0, 0, 0, 0, 0, 0, -1])
beq = np.array([0, 0, 0, 0, 0, 0])
max_q = [7, 1, 2, 3, 2, 1, 2, None]
bounds = tuple([(0, max_q[arcs]) for arcs in range(0, Aeq.shape[1])])

#optimization:
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')

#printing the solution:
usage = get_usage_string(arc_idxs, res.x.astype(int), max_q)
min_cut = get_min_cut(arc_idxs, res.x, np.array(max_q))
max_flow = res.fun * -1
print('## Results ## \n'
      'The usage of each arc will be (from, to): %s \n'
      'The arcs that produce the minimum cut (from, to): %s \n'
      'The maximum flow will be: %0.2f ' % (usage, min_cut , max_flow))

## Optimizer inputs ## 
Cost vector: [ 0  0  0  0  0  0  0 -1] 
 A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0  0 -1]
 [-1  0  1  1  0  0  0  0]
 [ 0 -1  0  0  1  0  0  0]
 [ 0  0 -1  0  0  1  0  0]
 [ 0  0  0  0 -1  0  1  0]
 [ 0  0  0 -1  0 -1 -1  1]] 
b_eq demand-supply vector: [0 0 0 0 0 0] 
Bounds of each X arc variable: ((0, 7), (0, 1), (0, 2), (0, 3), (0, 2), (0, 1), (0, 2), (0, None)) 

## Results ## 
The usage of each arc will be (from, to): {(0, 1): '4/7', (0, 2): '1/1', (1, 3): '1/2', (1, 5): '3/3', (2, 4): '1/2', (3, 5): '1/1', (4, 5): '1/2', (5, 0): '5/None'} 
The arcs that produce the minimum cut (from, to): [(0, 2), (1, 5), (3, 5)] 
The maximum flow will be: 5.00 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
